#### NLP | MVP

# Coronavirus Tweets: Spring 2020<a id='top'></a> 

# 

## **Analysis Goal**  
The client, [name](link), ...
wants to ... and 
requested a model to find a pattern ... in tweets in April. 

Impetus?

**RQ:** ....?

## **Process**
**Data source:** 
Coronavirus COVID-19 Tweets [early](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-early-april) and [late](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-late-april) April


**Models:** 
* [name] model, fit a [model name](#??):
    * eval metric
* [name] model, fit a [model name](#??):
    * eval metric


## **Preliminary Visualizations**
[Figure 1. Name](#??)<br/>
[Figure 1. Name](#??)<br/>

## **Preliminary Conclusions**
The [name] model ... showed....<br/>

The next step .... 

In [1]:
import pandas as pd
import glob 
pd.set_option('display.max_colwidth', None)


## 1 | Dataset: Coronavirus Tweets ([early](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-early-april) and [late](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-late-april) April) <a id='1'></a>  

In [16]:
# import one csv file to see data
single_df = pd.read_csv('20200401.csv')

In [22]:
single_df.head(2)

,status_id,user_id,created_at,screen_name,text,source,reply_to_status_id,reply_to_user_id,reply_to_screen_name,is_quote,...,retweet_count,country_code,place_full_name,place_type,followers_count,friends_count,account_lang,account_created_at,verified,lang
0,1245138808619724800,2722502906,2020-04-01T00:00:00Z,GradaNorteMX,"Cuando mejor iban las cosas en el circuito de tenis universitario de Estados Unidos, el #sonorense Alán Rubio volvió a Hermosillo ante la pandemia del #COVID_19 😕🇲🇽🇺🇸\n\n🎾 https://t.co/SldPvrP81A https://t.co/K7BVA6LV94",TweetDeck,NaN,NaN,NaN,False,...,0,NaN,NaN,NaN,1847,252,NaN,2014-08-10T21:20:32Z,False,es
1,1245138810071142405,817072420947247104,2020-04-01T00:00:00Z,Tu_IMSS_Coah,"El #Coronavirus se transmite de una persona infectada a otras a través de gotitas de saliva, acata las reglas de etiqueta | #EnfermedadesRespiratorias #PrevenciónCoronavirus \n#QuédateEnCasa #COVID19 #SanaDistancia\n#MéxicoUnido #IMSSolidario https://t.co/3F3rGiXjQm",TweetDeck,NaN,NaN,NaN,False,...,2,NaN,NaN,NaN,1576,169,NaN,2017-01-05T18:17:00Z,False,es


In [21]:
single_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591480 entries, 0 to 591479
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status_id             591480 non-null  int64  
 1   user_id               591480 non-null  int64  
 2   created_at            591480 non-null  object 
 3   screen_name           591480 non-null  object 
 4   text                  591480 non-null  object 
 5   source                591478 non-null  object 
 6   reply_to_status_id    69502 non-null   float64
 7   reply_to_user_id      84914 non-null   float64
 8   reply_to_screen_name  84914 non-null   object 
 9   is_quote              591480 non-null  bool   
 10  is_retweet            591480 non-null  bool   
 11  favourites_count      591480 non-null  int64  
 12  retweet_count         591480 non-null  int64  
 13  country_code          26040 non-null   object 
 14  place_full_name       26116 non-null   object 
 15  

In [24]:
# keep only the following 7 columns on full data import 

 2   created_at            591480 non-null  object 
 3   screen_name           591480 non-null  object 
 4   text                  591480 non-null  object 
 13  country_code          26040 non-null   object 
 18  account_lang          0 non-null       float64
 20  verified              591480 non-null  bool   
 21  lang                  591480 non-null  object 

In [4]:
# read list of April 2020 files, while ommitting columns above

path = r'/Users/sandraparedes/Documents/GitHub/metis_dsml/05_nlp/g00-nlp-project/data' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, usecols=[2,3,4,13,18,20,21])
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)


In [11]:
df.head()

,created_at,screen_name,text,country_code,account_lang,verified,lang
0,2020-04-06T00:00:00Z,EricSchneiderMD,The Pearl Harbor metaphor for #Covid_19 is an odd choice. Pearl Harbor was the start of 3 years of US mobilizing after trying to stay out of WWII.,NaN,NaN,False,en
1,2020-04-06T00:00:00Z,kubofinanciero,🟩 Para nosotros cada celular o computadora es una sucursal de kubo.financiero. Inicia un plan de inversión o pide un préstamo desde tu casa #ConectemosAúnMás 👉 https://t.co/p3eaYnigMZ #kubofinanciero #kubo #fintech #COVID19 https://t.co/xdIvsvXL5x,NaN,NaN,False,es
2,2020-04-06T00:00:00Z,CorpoelecAragu2,|| Paso a paso en centros de salud públicos y privados || #Covid19 #QuédateEnCasa https://t.co/AFxAtqXlEX,NaN,NaN,False,es
3,2020-04-06T00:00:00Z,Jaime_BonillaV,"Porque #EnBCNosCuidamosTodos, con la responsabilidad de nuestras acciones de distanciamiento social vamos a vencer al Covid-19.\n\n#EnBCNosCuidamosTodos #Coronavirus #COVID19 #CoronavirusBC #COVID19BC #QuédateEnCasa https://t.co/o9A2vdWldZ",NaN,NaN,True,es
4,2020-04-06T00:00:00Z,GobCDMX,"¿En tu casa hay personas adultas mayores, personas con discapacidad, mujeres embarazadas, niñas o niños?\n\nSigue estas recomendaciones para evitar accidentes durante esta emergencia sanitaria por #COVID19.\n\n#QuédateEnCasaYa 🏡 #QuédateEnCasaCDMX https://t.co/WTMpoE6QPD",NaN,NaN,True,es


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12786782 entries, 0 to 12786781
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   created_at    object 
 1   screen_name   object 
 2   text          object 
 3   country_code  object 
 4   account_lang  float64
 5   verified      bool   
 6   lang          object 
dtypes: bool(1), float64(1), object(5)
memory usage: 597.5+ MB


## 2 | Exploratory Data Analysis<a id='2'></a>  

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12786782 entries, 0 to 12786781
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   created_at    object 
 1   screen_name   object 
 2   text          object 
 3   country_code  object 
 4   account_lang  float64
 5   verified      bool   
 6   lang          object 
dtypes: bool(1), float64(1), object(5)
memory usage: 597.5+ MB


In [26]:
df['country_code'].unique()

array([nan, 'ZA', 'US', 'PT', 'PK', 'IN', 'BR', 'IT', 'TR', 'MX', 'BE',
       'GB', 'LK', 'DE', 'GT', 'TH', 'CL', 'DO', 'UY', 'AR', 'CA', 'SV',
       'CU', 'JP', 'CO', 'NG', 'ES', 'PY', 'PH', 'SA', 'KE', 'NZ', 'PA',
       'MY', 'JO', 'RS', 'AU', 'EC', 'HN', 'PL', 'SG', 'VE', 'PE', 'TT',
       'NI', 'CN', 'OM', 'MV', 'ID', 'JM', 'IE', 'GH', 'LV', 'KH', 'BO',
       'IL', 'NL', 'SN', 'GU', 'TW', 'NP', 'FJ', 'PF', 'UG', 'VN', 'BN',
       'LY', 'IR', 'CD', 'HK', 'BD', 'GN', 'FR', 'DK', 'CR', 'AE', 'HT',
       'HU', 'RU', 'KR', 'CH', 'IQ', 'TL', 'BH', 'PG', 'KZ', 'MA', 'KW',
       'PR', 'SE', 'GR', 'AT', 'SZ', 'FI', 'RW', 'QA', 'GE', 'NO', 'CM',
       'ET', 'MQ', 'BS', 'SC', 'LC', 'CZ', 'TZ', 'LB', 'MZ', 'EG', 'UA',
       'SO', 'ZW', 'CY', 'SL', 'MU', 'SY', 'BA', 'NC', 'BJ', 'RE', 'GP',
       'CG', 'GG', 'AF', 'LS', 'ZM', 'BW', 'AZ', 'AO', 'MW', 'ME', 'RO',
       'MT', 'HR', 'IS', 'SR', 'MN', 'AD', 'CI', 'IM', 'BY', 'DZ', 'BI',
       'MC', 'GQ', 'TG', 'AL', 'GI', 'ML', 'TN', 'MM

In [27]:
df['lang'].unique()

array(['en', 'es', 'in', 'th', 'und', 'pt', 'tl', 'ar', 'it', 'uk', 'ca',
       'hu', 'fr', 'ja', 'ko', 'el', 'tr', 'fi', 'de', 'ru', 'lt', 'zh',
       'eu', 'fa', 'nl', 'ur', 'bn', 'sv', 'hi', 'ro', 'ht', 'ckb', 'vi',
       'pl', 'sl', 'ta', 'ne', 'et', 'cy', 'da', 'si', 'ps', 'lv', 'cs',
       'iw', 'mr', 'ml', 'kn', 'te', 'no', 'or', 'gu', 'pa', 'am', 'lo',
       'dv', 'sr', 'is', 'ka', 'sd', 'bg', 'my', 'hy', 'km', 'bo', 'ug'],
      dtype=object)

In [28]:
print('English entries:', (df[df["lang"] == 'en'].count())['lang'])

English entries: 7128121


In [29]:
print('US entries:', (df[df["country_code"] == 'US'].count())['country_code'])

US entries: 153660


In [30]:
eng_us_df = df[(df['country_code'] == 'US') &
               (df['lang'] == 'en')]
eng_us_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 138789 entries, 141 to 12786770
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   created_at    138789 non-null  object 
 1   screen_name   138788 non-null  object 
 2   text          138789 non-null  object 
 3   country_code  138789 non-null  object 
 4   account_lang  0 non-null       float64
 5   verified      138789 non-null  bool   
 6   lang          138789 non-null  object 
dtypes: bool(1), float64(1), object(5)
memory usage: 7.5+ MB


### English + US df: 138,789 entries

In [31]:
eng_us_df.head(10)

,created_at,screen_name,text,country_code,account_lang,verified,lang
141,2020-04-06T00:00:05Z,WFMGINC,....#SUNDAYFUNDAY #coronavirus style #vino cheers 🍷 https://t.co/SrymChBkq2,US,NaN,False,en
234,2020-04-06T00:00:14Z,jpomietlasz,"This pandemic has confirmed my worst fears, most people don’t know how to make entertaining videos. #Covid_19 #SinceIveBeenQuarantined #AmericasUnfunniestVideos #WrestleMania #tonyaharding",US,NaN,False,en
263,2020-04-06T00:00:17Z,harold0719,Is this true? \nhttps://t.co/Xj2zdFe7ab\n#EcuadorEnEmergencia #Covid_19 #CoronavirusPandemic,US,NaN,False,en
273,2020-04-06T00:00:18Z,uche_blackstock,"Many us thought it was Wuhan province, but it could never be us. Then it was Italy, but it could never be us. Now it is here. One #NewYorker died every 12 minutes from #COVID19 over this weekend. Absolutely devastating.\n\nhttps://t.co/h5woMTfSCH",US,NaN,True,en
276,2020-04-06T00:00:18Z,BrentALang,Ah #coronavirus humor https://t.co/1yXAZXNIgz,US,NaN,True,en
278,2020-04-06T00:00:18Z,grantstern,Miami and South Florida in general are also staying home.\n\nNorth Florida thinks their immune to #COVID19. https://t.co/OiPfVqyMvC,US,NaN,True,en
290,2020-04-06T00:00:20Z,Thom_Cordeiro,How can President Trump be flip about a question about continuity of power and contracting the #coronavirus? “When I am around him I don’t breathe”? And we wonder why other Americans question and defy the quarantine. #CoronavirusTaskForce,US,NaN,False,en
322,2020-04-06T00:00:24Z,shelbalynn17,"This is our most desperate hour. Help us, Obi-Wan Kenobi. You're our only hope. #Covid_19 #StarWars #weneedaleader",US,NaN,False,en
329,2020-04-06T00:00:25Z,LheimIndustries,Face your fears....\n#business #love #coronavirus #covid19 #nyc #ny #la #California #zoom #happy #prayer https://t.co/idDV3bpkUr,US,NaN,False,en
340,2020-04-06T00:00:28Z,Elliot_Fuchs,"First Defense Nasal Screen coming up on @ABCSharkTank during the #Covid_19 outbreak and all of a sudden the guy doesn’t seem crazy.\n\nSharks redeemed themselves w/ mega-offers.\n\nPost Corona-airing, @mcuban @TheSharkDaymond @kevinolearytv is gonna make a killing. Replaces mask-need",US,NaN,False,en


In [9]:
eng_us_df['text'].head(10)

141                                                                                                                                                                                                                     ....#SUNDAYFUNDAY #coronavirus style #vino cheers 🍷 https://t.co/SrymChBkq2
234                                                                                                    This pandemic has confirmed my worst fears, most people don’t know how to make entertaining videos. #Covid_19 #SinceIveBeenQuarantined #AmericasUnfunniestVideos #WrestleMania #tonyaharding
263                                                                                                                                                                                                    Is this true? \nhttps://t.co/Xj2zdFe7ab\n#EcuadorEnEmergencia #Covid_19 #CoronavirusPandemic
273                                          Many us thought it was Wuhan province, but it could never be us. Then it was It

In [32]:
# #save cleaned df as tweet_df

# tweet_df = eng_us_df 
# tweet_df.to_pickle('coronavirus_tweets_df.pkl')
# tweet_df.to_csv(r'//Users/sandraparedes/Documents/GitHub/metis_dsml/05_nlp/g00-nlp-project/data/coronavirus_tweets_df.csv', index=False)

#read df in
df = pd.read_csv('coronavirus_tweets_df.csv')